In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import os

In [3]:
print(os.getcwd())

D:\Y3S1\IRWA_project\projectK\projectK


In [4]:
products = pd.read_csv('Dataset.csv')

In [5]:
products.head()

,Uniq Id,Crawl Timestamp,Dataset Origin,Product Id,Product Barcode,Product Company Type Source,Product Brand Source,Product Brand Normalised Source,Product Name Source,Match Rank,...,Product Currency,Product Available Inventory,Product Image Url,Product Model Number,Product Tags,Product Contents,Product Rating,Product Reviews Count,Bsr,Joining Key
0,1705736792d82aa2f2d3caf1c07c53f4,2020-09-24 03:21:12 +0000,NaN,2e17bf4acecdece67fc00f07ad62c910,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,NaN,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",NaN,NaN,NaN,NaN,81350af1be98d3753cf964709f0c766a
1,95a9fe6f4810fcfc7ff244fd06784f11,2020-10-30 14:04:08 +0000,NaN,076e5854a62dd283c253d6bae415af1f,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/9c8e42e4-13a5...,NaN,"Nice 'n Easy Permanent Color, 111 Natural Medi...",NaN,NaN,NaN,NaN,0353e63907dc0de0c734db4690300057
2,8d4d0330178d3ed181b15a4102b287f2,2020-08-06 05:51:47 +0000,NaN,8a4fe5d9c7a6ed26cc44d785a454b124,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,NaN,Clairol Nice 'N Easy Permanent Color 7/106A Na...,NaN,4.5,29221.0,NaN,b6985c8e94815fbca2319dbb8bf228af
3,fddc4df45b35efd886794b261f730c51,2020-07-15 11:22:04 +0000,NaN,03b5fb878a33eadff8b033419eab9669,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/25b4b467-bc61...,NaN,"Kokie Professional Matte Lipstick, Hot Berry, ...",NaN,NaN,NaN,NaN,85b70fded09186f00467cea2f935b779
4,0990cf89a59ca6a0460349a3e4f51d42,2020-11-26T12:27:20+00:00,NaN,ce3d761e57d6ccad80619297b5b1bcbc,NaN,Competitor,NaN,NaN,NaN,NaN,...,USD,111111111,https://i5.walmartimages.com/asr/1a2ebb06-cd01...,NaN,"Gillette TRAC II Plus Razor Blade Refills, Fit...",NaN,NaN,131.0,NaN,41c870871328e97da6fb036bb7d4b2da


In [6]:
products.shape

(5000, 32)

In [7]:
products.iloc[0]

Uniq Id                                             1705736792d82aa2f2d3caf1c07c53f4
Crawl Timestamp                                            2020-09-24 03:21:12 +0000
Dataset Origin                                                                   NaN
Product Id                                          2e17bf4acecdece67fc00f07ad62c910
Product Barcode                                                                  NaN
Product Company Type Source                                               Competitor
Product Brand Source                                                             NaN
Product Brand Normalised Source                                                  NaN
Product Name Source                                                              NaN
Match Rank                                                                       NaN
Match Score                                                                      NaN
Match Type                                                       

In [8]:
products.columns

Index(['Uniq Id', 'Crawl Timestamp', 'Dataset Origin', 'Product Id',
       'Product Barcode', 'Product Company Type Source',
       'Product Brand Source', 'Product Brand Normalised Source',
       'Product Name Source', 'Match Rank', 'Match Score', 'Match Type',
       'Retailer', 'Product Category', 'Product Brand', 'Product Name',
       'Product Price', 'Sku', 'Upc', 'Product Url', 'Market',
       'Product Description', 'Product Currency',
       'Product Available Inventory', 'Product Image Url',
       'Product Model Number', 'Product Tags', 'Product Contents',
       'Product Rating', 'Product Reviews Count', 'Bsr', 'Joining Key'],
      dtype='object')

In [9]:
train_data = products[['Uniq Id', 'Product Id', 'Product Rating', 'Product Reviews Count', 'Product Category', 'Product Brand', 'Product Name', 'Product Image Url', 'Product Description', 'Product Tags']]

In [10]:
train_data.head(2)

,Uniq Id,Product Id,Product Rating,Product Reviews Count,Product Category,Product Brand,Product Name,Product Image Url,Product Description,Product Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2e17bf4acecdece67fc00f07ad62c910,NaN,NaN,Premium Beauty > Premium Makeup > Premium Nail...,OPI,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,NaN,"OPI Infinite Shine, Nail Lacquer Nail Polish, ..."
1,95a9fe6f4810fcfc7ff244fd06784f11,076e5854a62dd283c253d6bae415af1f,NaN,NaN,Beauty > Hair Care > Hair Color > Auburn Hair ...,Nice'n Easy,"Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,"Nice 'n Easy Permanent Color, 111 Natural Medi..."


In [11]:
train_data.isna().sum()

Uniq Id                     0
Product Id                  0
Product Rating           2806
Product Reviews Count    1654
Product Category           10
Product Brand              13
Product Name                0
Product Image Url           0
Product Description      1127
Product Tags                0
dtype: int64

In [12]:
#train_data.dropna(inplace=True)

In [13]:
#filling missing values
train_data['Product Rating'].fillna(0, inplace=True)
train_data['Product Reviews Count'].fillna(0, inplace=True)
train_data['Product Category'].fillna('No Category', inplace=True)
train_data['Product Brand'].fillna('No Brand', inplace=True)
train_data['Product Description'].fillna('No Description', inplace=True)

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\4023703667.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Product Rating'].fillna(0, inplace=True)
C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\4023703667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Product Rating'].fillna(0, inplace=True)
C:\Users\Kabi\App

In [14]:
train_data.isna().sum()

Uniq Id                  0
Product Id               0
Product Rating           0
Product Reviews Count    0
Product Category         0
Product Brand            0
Product Name             0
Product Image Url        0
Product Description      0
Product Tags             0
dtype: int64

In [15]:
train_data.duplicated().sum()

0

In [16]:
train_data.shape

(5000, 10)

In [17]:
# make columns shorter
# Define the mapping of current column names to shorter names
column_name_mapping = {
    'Uniq Id': 'ID',
    'Product Id': 'ProdID',
    'Product Rating': 'Rating',
    'Product Reviews Count': 'ReviewCount',
    'Product Category': 'Category',
    'Product Brand': 'Brand',
    'Product Name': 'Name',
    'Product Image Url': 'ImageURL',
    'Product Description': 'Description',
    'Product Tags': 'Tags',
    'Product Contents': 'Contents'
}
# Rename the columns using the mapping
train_data.rename(columns=column_name_mapping, inplace=True)

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\2411302344.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.rename(columns=column_name_mapping, inplace=True)


In [18]:
train_data.head(2)

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2e17bf4acecdece67fc00f07ad62c910,0.0,0.0,Premium Beauty > Premium Makeup > Premium Nail...,OPI,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,No Description,"OPI Infinite Shine, Nail Lacquer Nail Polish, ..."
1,95a9fe6f4810fcfc7ff244fd06784f11,076e5854a62dd283c253d6bae415af1f,0.0,0.0,Beauty > Hair Care > Hair Color > Auburn Hair ...,Nice'n Easy,"Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,"Nice 'n Easy Permanent Color, 111 Natural Medi..."


In [19]:
#convert id and product id into float
# train_data['ID'] = train_data['ID'].astype(str).str.extract('(\d+)').astype('float') # Use int64 to handle large integers
train_data['ProdID'] = train_data['ProdID'].astype(str).str.extract('(\d+)').astype('float') # Use int64 to handle large integers

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\2966118085.py:3: SyntaxWarning: invalid escape sequence '\d'
  train_data['ProdID'] = train_data['ProdID'].astype(str).str.extract('(\d+)').astype('float') # Use int64 to handle large integers
C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\2966118085.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['ProdID'] = train_data['ProdID'].astype(str).str.extract('(\d+)').astype('float') # Use int64 to handle large integers


In [20]:
train_data.head()

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2.0,0.0,0.0,Premium Beauty > Premium Makeup > Premium Nail...,OPI,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,No Description,"OPI Infinite Shine, Nail Lacquer Nail Polish, ..."
1,95a9fe6f4810fcfc7ff244fd06784f11,76.0,0.0,0.0,Beauty > Hair Care > Hair Color > Auburn Hair ...,Nice'n Easy,"Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,"Nice 'n Easy Permanent Color, 111 Natural Medi..."
2,8d4d0330178d3ed181b15a4102b287f2,8.0,4.5,29221.0,Beauty > Hair Care > Hair Color > Permanent Ha...,Clairol,Clairol Nice N Easy Permanent Color 7/106A Nat...,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,This Clairol Nice N Easy Permanent Color gives...,Clairol Nice 'N Easy Permanent Color 7/106A Na...
3,fddc4df45b35efd886794b261f730c51,3.0,0.0,0.0,Beauty > Makeup > Lip,Kokie Cosmetics,"Kokie Professional Matte Lipstick, Hot Berry, ...",https://i5.walmartimages.com/asr/25b4b467-bc61...,Calling all matte lip lovers! Indulge in our r...,"Kokie Professional Matte Lipstick, Hot Berry, ..."
4,0990cf89a59ca6a0460349a3e4f51d42,3.0,0.0,131.0,Seasonal > Stock Up Essentials > Personal Care...,Gillette,"Gillette TRAC II Plus Razor Blade Refills, Fit...",https://i5.walmartimages.com/asr/1a2ebb06-cd01...,"In 1971, Gillette introduced the Trac II razor...","Gillette TRAC II Plus Razor Blade Refills, Fit..."


In [21]:
#Spliting category column
train_data['Category'] = train_data['Category'].apply(lambda x: x.replace(" ", ","))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3503757238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Category'] = train_data['Category'].apply(lambda x: x.replace(" ", ","))


In [22]:
train_data['Category'] = train_data['Category'].apply(lambda x: x.replace(">,", ""))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\4033171402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Category'] = train_data['Category'].apply(lambda x: x.replace(">,", ""))


In [23]:
train_data.iloc[0]['Category']

'Premium,Beauty,Premium,Makeup,Premium,Nail,Polish,&,Care,Premium,Nail,Polish'

In [24]:
train_data['Category'] = train_data['Category'].apply(lambda x:x.lower())

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3936269111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Category'] = train_data['Category'].apply(lambda x:x.lower())


In [25]:
train_data.iloc[0]['Category']

'premium,beauty,premium,makeup,premium,nail,polish,&,care,premium,nail,polish'

In [26]:
from posixpath import split
train_data['Category'] = train_data['Category'].apply(lambda x:x.split(","))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\927357734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Category'] = train_data['Category'].apply(lambda x:x.split(","))


In [27]:
train_data.iloc[0]['Category']

['premium',
 'beauty',
 'premium',
 'makeup',
 'premium',
 'nail',
 'polish',
 '&',
 'care',
 'premium',
 'nail',
 'polish']

In [28]:
#splitting brand column
train_data.iloc[5]['Brand']

'Old Spice'

In [29]:
train_data.iloc[12]['Brand']

"L'Oreal Paris"

In [30]:
train_data['Brand'] = train_data['Brand'].apply(lambda x:x.replace(","," "))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3933393050.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Brand'] = train_data['Brand'].apply(lambda x:x.replace(","," "))


In [31]:
train_data['Brand'] = train_data['Brand'].apply(lambda x:x.replace("'"," "))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3436795443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Brand'] = train_data['Brand'].apply(lambda x:x.replace("'"," "))


In [32]:
train_data['Brand'] = train_data['Brand'].apply(lambda x:x.lower())

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3343635098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Brand'] = train_data['Brand'].apply(lambda x:x.lower())


In [33]:
from posixpath import split
train_data['Brand'] = train_data['Brand'].apply(lambda x:x.split(" "))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3095049565.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Brand'] = train_data['Brand'].apply(lambda x:x.split(" "))


In [34]:
train_data.iloc[5]['Brand']

['old', 'spice']

In [35]:
train_data.iloc[12]['Brand']

['l', 'oreal', 'paris']

In [36]:
train_data.head()

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2.0,0.0,0.0,"[premium, beauty, premium, makeup, premium, na...",[opi],"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,No Description,"OPI Infinite Shine, Nail Lacquer Nail Polish, ..."
1,95a9fe6f4810fcfc7ff244fd06784f11,76.0,0.0,0.0,"[beauty, hair, care, hair, color, auburn, hair...","[nice, n, easy]","Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,Pack of 3 Pack of 3 for the UPC: 381519000201 ...,"Nice 'n Easy Permanent Color, 111 Natural Medi..."
2,8d4d0330178d3ed181b15a4102b287f2,8.0,4.5,29221.0,"[beauty, hair, care, hair, color, permanent, h...",[clairol],Clairol Nice N Easy Permanent Color 7/106A Nat...,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,This Clairol Nice N Easy Permanent Color gives...,Clairol Nice 'N Easy Permanent Color 7/106A Na...
3,fddc4df45b35efd886794b261f730c51,3.0,0.0,0.0,"[beauty, makeup, lip]","[kokie, cosmetics]","Kokie Professional Matte Lipstick, Hot Berry, ...",https://i5.walmartimages.com/asr/25b4b467-bc61...,Calling all matte lip lovers! Indulge in our r...,"Kokie Professional Matte Lipstick, Hot Berry, ..."
4,0990cf89a59ca6a0460349a3e4f51d42,3.0,0.0,131.0,"[seasonal, stock, up, essentials, personal, ca...",[gillette],"Gillette TRAC II Plus Razor Blade Refills, Fit...",https://i5.walmartimages.com/asr/1a2ebb06-cd01...,"In 1971, Gillette introduced the Trac II razor...","Gillette TRAC II Plus Razor Blade Refills, Fit..."


In [37]:
#splitting Description
train_data.iloc[5]['Description']

'Old Spice Artisan Molding Clay is a Mens Styling product that allows men sculpt their hair into a moldable look with high hold and a matte finish. Old Spice Artisan Styling High Hold Matte Finish Molding Clay, 2.64 oz: Old Spice Artisan Molding Clay provides high hold and a matte finish Old Spice Artisan Molding Clay helps tame and shape your hair For a moldable look Available in one size: 75 g / 2.64 oz. puck'

In [38]:
train_data['Description'] = train_data['Description'].apply(lambda x:x.replace(",",""))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3517882156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Description'] = train_data['Description'].apply(lambda x:x.replace(",",""))


In [39]:
train_data['Description'] = train_data['Description'].apply(lambda x:x.lower())

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3965188533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Description'] = train_data['Description'].apply(lambda x:x.lower())


In [40]:
train_data.iloc[5]['Description']

'old spice artisan molding clay is a mens styling product that allows men sculpt their hair into a moldable look with high hold and a matte finish. old spice artisan styling high hold matte finish molding clay 2.64 oz: old spice artisan molding clay provides high hold and a matte finish old spice artisan molding clay helps tame and shape your hair for a moldable look available in one size: 75 g / 2.64 oz. puck'

In [41]:
from posixpath import split
train_data['Description']=train_data['Description'].apply(lambda x:x.split(" "))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\1433719083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Description']=train_data['Description'].apply(lambda x:x.split(" "))


In [42]:
train_data.iloc[5]['Description']

['old',
 'spice',
 'artisan',
 'molding',
 'clay',
 'is',
 'a',
 'mens',
 'styling',
 'product',
 'that',
 'allows',
 'men',
 'sculpt',
 'their',
 'hair',
 'into',
 'a',
 'moldable',
 'look',
 'with',
 'high',
 'hold',
 'and',
 'a',
 'matte',
 'finish.',
 'old',
 'spice',
 'artisan',
 'styling',
 'high',
 'hold',
 'matte',
 'finish',
 'molding',
 'clay',
 '2.64',
 'oz:',
 'old',
 'spice',
 'artisan',
 'molding',
 'clay',
 'provides',
 'high',
 'hold',
 'and',
 'a',
 'matte',
 'finish',
 'old',
 'spice',
 'artisan',
 'molding',
 'clay',
 'helps',
 'tame',
 'and',
 'shape',
 'your',
 'hair',
 'for',
 'a',
 'moldable',
 'look',
 'available',
 'in',
 'one',
 'size:',
 '75',
 'g',
 '/',
 '2.64',
 'oz.',
 'puck']

In [43]:
train_data.iloc[1]['Description']

['pack',
 'of',
 '3',
 'pack',
 'of',
 '3',
 'for',
 'the',
 'upc:',
 '381519000201',
 'beautiful',
 'natural',
 'color.',
 'active',
 'ingredients:',
 'color-blend',
 'formula:',
 'water;',
 'ethoxydiglycol;',
 'propylene',
 'glycol;',
 'isopropyl',
 'alcohol;',
 'soytrimonium',
 'chloride;',
 'oleth-5;',
 'oleic',
 'acid;',
 'ammonium',
 'hydroxide;',
 'trisodium',
 'ethylenediamine',
 'disuccinate;',
 'fragrance;',
 'oleth-2;',
 'c11-15',
 'pareth-9;',
 'cocamidopropyl',
 'betaine;',
 'c12-15',
 'pareth-3;',
 'erythorbic',
 'acid;',
 'citric',
 'acid;',
 'sodium',
 'sulfite;',
 'edta;',
 'p-aminophenol;',
 '4-amino-2-hydroxytoluene;',
 'p-phenylenediamine;',
 'resorcinol;',
 '2-methylresorcinol;',
 'nn-bis(2-hydroxyethyl)-p-phenylenediamine',
 'sulfate;',
 '1-hydroxyethyl',
 '45-diamino',
 'pyrazole',
 'sulfate;',
 'phenyl',
 'methyl',
 'pyrazolone;',
 '1-naphthol.',
 'color-blend',
 'activator:',
 'water;',
 'hydrogen',
 'peroxide;',
 'acrylates',
 'copolymer;',
 'steareth-21;',
 '

In [44]:
train_data.head(2)

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2.0,0.0,0.0,"[premium, beauty, premium, makeup, premium, na...",[opi],"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,"[no, description]","OPI Infinite Shine, Nail Lacquer Nail Polish, ..."
1,95a9fe6f4810fcfc7ff244fd06784f11,76.0,0.0,0.0,"[beauty, hair, care, hair, color, auburn, hair...","[nice, n, easy]","Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,"[pack, of, 3, pack, of, 3, for, the, upc:, 381...","Nice 'n Easy Permanent Color, 111 Natural Medi..."


In [45]:
#spliting tag
train_data.iloc[0]['Tags']

'OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath, Wal-mart, Walmart.com'

In [46]:
train_data['Tags'] = train_data['Tags'].apply(lambda x:x.replace(",",""))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\2704313025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Tags'] = train_data['Tags'].apply(lambda x:x.replace(",",""))


In [47]:
train_data['Tags'] = train_data['Tags'].apply(lambda x:x.lower())

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\1739430253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Tags'] = train_data['Tags'].apply(lambda x:x.lower())


In [48]:
from posixpath import split
train_data['Tags'] = train_data['Tags'].apply(lambda x:x.split(" "))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3867102203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Tags'] = train_data['Tags'].apply(lambda x:x.split(" "))


In [49]:
train_data.iloc[0]['Tags']

['opi',
 'infinite',
 'shine',
 'nail',
 'lacquer',
 'nail',
 'polish',
 'bubble',
 'bath',
 'wal-mart',
 'walmart.com']

In [50]:
train_data.head(2)

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2.0,0.0,0.0,"[premium, beauty, premium, makeup, premium, na...",[opi],"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,"[no, description]","[opi, infinite, shine, nail, lacquer, nail, po..."
1,95a9fe6f4810fcfc7ff244fd06784f11,76.0,0.0,0.0,"[beauty, hair, care, hair, color, auburn, hair...","[nice, n, easy]","Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,"[pack, of, 3, pack, of, 3, for, the, upc:, 381...","[nice, 'n, easy, permanent, color, 111, natura..."


In [51]:
def remove_space(text):
    i = []
    for j in text:
        i.append(j.replace(" ",""))
    return i

In [52]:
train_data['Brand'] = train_data['Brand'].apply(remove_space)

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\2916315606.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Brand'] = train_data['Brand'].apply(remove_space)


In [53]:
train_data.head()

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags
0,1705736792d82aa2f2d3caf1c07c53f4,2.0,0.0,0.0,"[premium, beauty, premium, makeup, premium, na...",[opi],"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,"[no, description]","[opi, infinite, shine, nail, lacquer, nail, po..."
1,95a9fe6f4810fcfc7ff244fd06784f11,76.0,0.0,0.0,"[beauty, hair, care, hair, color, auburn, hair...","[nice, n, easy]","Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,"[pack, of, 3, pack, of, 3, for, the, upc:, 381...","[nice, 'n, easy, permanent, color, 111, natura..."
2,8d4d0330178d3ed181b15a4102b287f2,8.0,4.5,29221.0,"[beauty, hair, care, hair, color, permanent, h...",[clairol],Clairol Nice N Easy Permanent Color 7/106A Nat...,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,"[this, clairol, nice, n, easy, permanent, colo...","[clairol, nice, 'n, easy, permanent, color, 7/..."
3,fddc4df45b35efd886794b261f730c51,3.0,0.0,0.0,"[beauty, makeup, lip]","[kokie, cosmetics]","Kokie Professional Matte Lipstick, Hot Berry, ...",https://i5.walmartimages.com/asr/25b4b467-bc61...,"[calling, all, matte, lip, lovers!, indulge, i...","[kokie, professional, matte, lipstick, hot, be..."
4,0990cf89a59ca6a0460349a3e4f51d42,3.0,0.0,131.0,"[seasonal, stock, up, essentials, personal, ca...",[gillette],"Gillette TRAC II Plus Razor Blade Refills, Fit...",https://i5.walmartimages.com/asr/1a2ebb06-cd01...,"[in, 1971, gillette, introduced, the, trac, ii...","[gillette, trac, ii, plus, razor, blade, refil..."


In [54]:
#create new column and concetanate other columns
train_data['ProdTags'] = train_data['Category']+train_data['Brand']+train_data['Description']+train_data['Tags']

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3467879121.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['ProdTags'] = train_data['Category']+train_data['Brand']+train_data['Description']+train_data['Tags']


In [55]:
train_data.head()

,ID,ProdID,Rating,ReviewCount,Category,Brand,Name,ImageURL,Description,Tags,ProdTags
0,1705736792d82aa2f2d3caf1c07c53f4,2.0,0.0,0.0,"[premium, beauty, premium, makeup, premium, na...",[opi],"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",https://i5.walmartimages.com/asr/0e1f4c51-c1a4...,"[no, description]","[opi, infinite, shine, nail, lacquer, nail, po...","[premium, beauty, premium, makeup, premium, na..."
1,95a9fe6f4810fcfc7ff244fd06784f11,76.0,0.0,0.0,"[beauty, hair, care, hair, color, auburn, hair...","[nice, n, easy]","Nice n Easy Permanent Color, 111 Natural Mediu...",https://i5.walmartimages.com/asr/9c8e42e4-13a5...,"[pack, of, 3, pack, of, 3, for, the, upc:, 381...","[nice, 'n, easy, permanent, color, 111, natura...","[beauty, hair, care, hair, color, auburn, hair..."
2,8d4d0330178d3ed181b15a4102b287f2,8.0,4.5,29221.0,"[beauty, hair, care, hair, color, permanent, h...",[clairol],Clairol Nice N Easy Permanent Color 7/106A Nat...,https://i5.walmartimages.com/asr/e3a601c2-6a2b...,"[this, clairol, nice, n, easy, permanent, colo...","[clairol, nice, 'n, easy, permanent, color, 7/...","[beauty, hair, care, hair, color, permanent, h..."
3,fddc4df45b35efd886794b261f730c51,3.0,0.0,0.0,"[beauty, makeup, lip]","[kokie, cosmetics]","Kokie Professional Matte Lipstick, Hot Berry, ...",https://i5.walmartimages.com/asr/25b4b467-bc61...,"[calling, all, matte, lip, lovers!, indulge, i...","[kokie, professional, matte, lipstick, hot, be...","[beauty, makeup, lip, kokie, cosmetics, callin..."
4,0990cf89a59ca6a0460349a3e4f51d42,3.0,0.0,131.0,"[seasonal, stock, up, essentials, personal, ca...",[gillette],"Gillette TRAC II Plus Razor Blade Refills, Fit...",https://i5.walmartimages.com/asr/1a2ebb06-cd01...,"[in, 1971, gillette, introduced, the, trac, ii...","[gillette, trac, ii, plus, razor, blade, refil...","[seasonal, stock, up, essentials, personal, ca..."


In [56]:
train_data.iloc[0]['ProdTags']

['premium',
 'beauty',
 'premium',
 'makeup',
 'premium',
 'nail',
 'polish',
 '&',
 'care',
 'premium',
 'nail',
 'polish',
 'opi',
 'no',
 'description',
 'opi',
 'infinite',
 'shine',
 'nail',
 'lacquer',
 'nail',
 'polish',
 'bubble',
 'bath',
 'wal-mart',
 'walmart.com']

In [57]:
#Conert to string as paragraph again
new_df = train_data[['ID','Name','ProdTags']]

In [58]:
new_df.head()

,ID,Name,ProdTags
0,1705736792d82aa2f2d3caf1c07c53f4,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...","[premium, beauty, premium, makeup, premium, na..."
1,95a9fe6f4810fcfc7ff244fd06784f11,"Nice n Easy Permanent Color, 111 Natural Mediu...","[beauty, hair, care, hair, color, auburn, hair..."
2,8d4d0330178d3ed181b15a4102b287f2,Clairol Nice N Easy Permanent Color 7/106A Nat...,"[beauty, hair, care, hair, color, permanent, h..."
3,fddc4df45b35efd886794b261f730c51,"Kokie Professional Matte Lipstick, Hot Berry, ...","[beauty, makeup, lip, kokie, cosmetics, callin..."
4,0990cf89a59ca6a0460349a3e4f51d42,"Gillette TRAC II Plus Razor Blade Refills, Fit...","[seasonal, stock, up, essentials, personal, ca..."


In [59]:
new_df['ProdTags'] = new_df['ProdTags'].apply(lambda x:" ".join(x))

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\653642007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ProdTags'] = new_df['ProdTags'].apply(lambda x:" ".join(x))


In [60]:
new_df.head()

,ID,Name,ProdTags
0,1705736792d82aa2f2d3caf1c07c53f4,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",premium beauty premium makeup premium nail pol...
1,95a9fe6f4810fcfc7ff244fd06784f11,"Nice n Easy Permanent Color, 111 Natural Mediu...",beauty hair care hair color auburn hair color ...
2,8d4d0330178d3ed181b15a4102b287f2,Clairol Nice N Easy Permanent Color 7/106A Nat...,beauty hair care hair color permanent hair col...
3,fddc4df45b35efd886794b261f730c51,"Kokie Professional Matte Lipstick, Hot Berry, ...",beauty makeup lip kokie cosmetics calling all ...
4,0990cf89a59ca6a0460349a3e4f51d42,"Gillette TRAC II Plus Razor Blade Refills, Fit...",seasonal stock up essentials personal care sto...


In [61]:
new_df.iloc[0]['ProdTags']

'premium beauty premium makeup premium nail polish & care premium nail polish opi no description opi infinite shine nail lacquer nail polish bubble bath wal-mart walmart.com'

In [62]:
new_df.shape

(5000, 3)

In [63]:
import nltk
from nltk.stem import PorterStemmer

In [64]:
pStem = PorterStemmer()

In [65]:
#Stemmer function
def stemmers(text):
    i = []
    for j in text.split():
        i.append(pStem.stem(j))
    return " ".join(i)

In [66]:
new_df['ProdTags'] = new_df['ProdTags'].apply(stemmers)

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3213948483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['ProdTags'] = new_df['ProdTags'].apply(stemmers)


In [67]:
new_df.iloc[0]['ProdTags']

'premium beauti premium makeup premium nail polish & care premium nail polish opi no descript opi infinit shine nail lacquer nail polish bubbl bath wal-mart walmart.com'

In [68]:
#convert to count vectorizer

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [70]:
vector = cv.fit_transform(new_df['ProdTags']).toarray()

In [71]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [72]:
vector.shape

(5000, 5000)

In [73]:
#find cosine similarity

In [74]:
#from sklearn.metrics.pairwise import cosine_similarity

In [75]:
cos_sim = cosine_similarity(vector)

In [76]:
cos_sim

array([[1.        , 0.05443677, 0.03077525, ..., 0.4112356 , 0.41014625,
        0.13764654],
       [0.05443677, 1.        , 0.55398214, ..., 0.05375945, 0.0619602 ,
        0.10667355],
       [0.03077525, 0.55398214, 1.        , ..., 0.07786856, 0.08807176,
        0.1228471 ],
       ...,
       [0.4112356 , 0.05375945, 0.07786856, ..., 1.        , 0.03058678,
        0.09426721],
       [0.41014625, 0.0619602 , 0.08807176, ..., 0.03058678, 1.        ,
        0.12063588],
       [0.13764654, 0.10667355, 0.1228471 , ..., 0.09426721, 0.12063588,
        1.        ]])

In [77]:
cos_sim.shape

(5000, 5000)

In [78]:
new_df.iloc[0]['Name']

'OPI Infinite Shine, Nail Lacquer Nail Polish, Bubble Bath'

In [79]:
#recommendation

In [80]:
#I need to search by words if i give word it should display all the indexes that have the words i input.
# allo me to input the word

def search_by_words(word):
  """
  Searches for a word in the 'ProdTags' column and returns the indexes of matching rows.

  Args:
    word: The word to search for.

  Returns:
    A list of indexes where the word is found in the 'ProdTags' column.
  """
  indexes = []
  for index, row in new_df.iterrows():
    if word in row['ProdTags']:
      indexes.append(index)
  return indexes

# Get user input for the word to search
search_word = input("Enter the word to search: ")

# Search for the word and display the matching indexes
matching_indexes = search_by_words(search_word)

if matching_indexes:
  print(f"The word '{search_word}' was found in the following indexes:")
  print(matching_indexes)
else:
  print(f"The word '{search_word}' was not found in any product tags.")


Enter the word to search:  huggggqwe


The word 'huggggqwe' was not found in any product tags.


In [81]:
new_df[new_df['Name']=='Dry Conditioner'].index[0]

51

In [82]:
#Create recommend function according to above seach word code

def recommend(product):
  index = new_df[new_df['Name'] == product].index[0]
  distances = sorted(list(enumerate(cos_sim[index])),reverse=True,key = lambda x: x[1])
  for i in distances[1:6]:
    print(new_df.iloc[i[0]].Name)


In [83]:
recommend('Dry Conditioner')

TRESemme Moisture Rich Conditioner with Pump, 39 fl oz
Dove Nourishing Secrets Coconut & Hydration Conditioner with Pump, 31 oz
($23 Value) Carols Daughter Black Vanilla Conditioner Gift Set, Hydrating Conditioner 12 fl oz, Leave-In Conditioner 8 fl oz
Gliss Hair Repair Conditioner, Ultra Moisutre, 13.6 Ounce
Sexy Hair Concepts Healthy Sexy Hair Tri-Wheat Leave In Conditioner 8.5fl Oz


In [84]:
import pickle

pickle.dump(new_df,open('product_list.pkl','wb'))
pickle.dump(cos_sim,open('similarity.pkl','wb'))

In [85]:
recommend('OxiClean Odor Blasters')

Tide Washing Machine Cleaner 5ct
Tide Febreze Sport HE, 59 Loads Liquid Laundry Detergent, 92 fl oz
OxiClean Color Boost Color Brightener plus Stain Remover Power Paks, 18 Count
OxiClean White Revive Liquid Laundry Whitener + Stain Remover, 66oz
Clorox 2 for Colors - Stain Remover and Color Brightener, 88 Ounces


In [86]:
recommend('SmileFrida the ToothHugger')

Oral-B Kids Pro-Health Stages Jake & The Neverland Pirates Battery Electric Toothbrush
Colgate Kids Toothbrush, Toothpaste, Mouthwash Gift Set - Ryans World
Oral-B Kids Manual Toothbrush featuring Disneys Princess Characters, Soft Bristles, for Children and Toddlers 3+, 1 count
Pack of 5 Dr. Fresh Kids Extra Soft Toothbrushes (2 packs)
Firefly Ready Go My Little Pony Toothbrush


In [87]:
new_df.head()

,ID,Name,ProdTags
0,1705736792d82aa2f2d3caf1c07c53f4,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",premium beauti premium makeup premium nail pol...
1,95a9fe6f4810fcfc7ff244fd06784f11,"Nice n Easy Permanent Color, 111 Natural Mediu...",beauti hair care hair color auburn hair color ...
2,8d4d0330178d3ed181b15a4102b287f2,Clairol Nice N Easy Permanent Color 7/106A Nat...,beauti hair care hair color perman hair color ...
3,fddc4df45b35efd886794b261f730c51,"Kokie Professional Matte Lipstick, Hot Berry, ...",beauti makeup lip koki cosmet call all matt li...
4,0990cf89a59ca6a0460349a3e4f51d42,"Gillette TRAC II Plus Razor Blade Refills, Fit...",season stock up essenti person care stock up s...


In [88]:
new_df['imgURL']=train_data['ImageURL']

C:\Users\Kabi\AppData\Local\Temp\ipykernel_8476\3064075425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['imgURL']=train_data['ImageURL']


In [89]:
new_df.head()

,ID,Name,ProdTags,imgURL
0,1705736792d82aa2f2d3caf1c07c53f4,"OPI Infinite Shine, Nail Lacquer Nail Polish, ...",premium beauti premium makeup premium nail pol...,https://i5.walmartimages.com/asr/0e1f4c51-c1a4...
1,95a9fe6f4810fcfc7ff244fd06784f11,"Nice n Easy Permanent Color, 111 Natural Mediu...",beauti hair care hair color auburn hair color ...,https://i5.walmartimages.com/asr/9c8e42e4-13a5...
2,8d4d0330178d3ed181b15a4102b287f2,Clairol Nice N Easy Permanent Color 7/106A Nat...,beauti hair care hair color perman hair color ...,https://i5.walmartimages.com/asr/e3a601c2-6a2b...
3,fddc4df45b35efd886794b261f730c51,"Kokie Professional Matte Lipstick, Hot Berry, ...",beauti makeup lip koki cosmet call all matt li...,https://i5.walmartimages.com/asr/25b4b467-bc61...
4,0990cf89a59ca6a0460349a3e4f51d42,"Gillette TRAC II Plus Razor Blade Refills, Fit...",season stock up essenti person care stock up s...,https://i5.walmartimages.com/asr/1a2ebb06-cd01...


In [90]:
# Save the DataFrame as a CSV file
new_df.to_csv('new_dataset.csv', index=False)

In [91]:
#Disable the 'a' key shortcut for adding a new cell
require(["base/js/namespace"], function(Jupyter) {
    Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('a','d');
});

SyntaxError: invalid syntax (2130327721.py, line 2)